In [1]:
from PandaStation import PandaStation
from PandaStation import AddShape
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG



import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

In [2]:
ycb = [("cracker", "003_cracker_box.sdf"), 
        ("sugar", "004_sugar_box.sdf"), 
        ("soup", "005_tomato_soup_can.sdf"), 
        ("mustard", "006_mustard_bottle.sdf"), 
        ("gelatin", "009_gelatin_box.sdf"), 
        ("meat", "010_potted_meat_can.sdf")]

In [ ]:

#X_initial = RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0 + np.random.uniform(-1,1))
#                           , [-.1+ np.random.uniform(-0.1, 0.1), -.65 + np.random.uniform(-0.1, 0.1), 0.091])

X1 = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2.0)
                           ,[-.1, -.6, 0.2])

X = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2.0)
                           ,[-0.1, -.6, 0.2])

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation(time_step = 0.001))
station.SetupBinStation()
#station.AddManipulandFromFile(
#    "drake/manipulation/models/ycb/sdf/" + ycb[3][1],
#    X1)

station.AddModelFromFile(
    FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf"),
    X)


plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")


station.Finalize()


planner = builder.AddSystem(BinPointCloudToGraspSystem())
builder.Connect(station.GetOutputPort("camera0_point_cloud"), planner.GetInputPort("camera0_pcd"))
builder.Connect(station.GetOutputPort("camera1_point_cloud"), planner.GetInputPort("camera1_pcd"))
builder.Connect(station.GetOutputPort("camera2_point_cloud"), planner.GetInputPort("camera2_pcd"))
builder.Connect(station.GetOutputPort("panda_position_measured"), planner.GetInputPort("panda_position"))

builder.Connect(planner.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
builder.Connect(planner.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))



station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions


diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
#station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation



meshcat.start_recording()

simulator.AdvanceTo(37)

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6040...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7040/static/
Connected to meshcat-server.
INITIALIZATION
Try number 18
FOUND GRASP
TO PREPICK
PICKING
TO PREPLACE
PLACING
TO REST
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Ope

In [ ]:
#file = open("media/videos/stacking_blocksV1.html", "w")
#file.write(meshcat.vis.static_html())
#file.close()